In [24]:
import pandas as pd
import numpy as np

In [65]:
df = pd.read_csv('./ConvertRate.csv')
df

,buld_gbn,conv_rate,level_no_out,region_cd,region_nm,research_date
0,2,9.272727,3,1100N67,두정,201901
1,2,10.038462,3,1100N68,보령,201901
2,2,9.400000,3,1100Y19N012,합덕버스터미널,201901
3,2,10.000000,3,1200N73,김제,201901
4,2,7.500000,3,1300N78,광양중동,201901
...,...,...,...,...,...,...
10545,3,5.989286,3,4800127,거제옥포,202202
10546,3,6.200000,3,4800117,김해시 내외동,202202
10547,3,5.618462,3,4800N241,마산동서동,202202
10548,3,6.063253,3,4800121,창원시청,202202


In [6]:
df.tail()

,BULD_GBN,INCOME_RATE,LEVEL_NO_OUT,REGION_CD,REGION_NM,RESEARCH_DATE
7069,2,2.630401,3,0300311,계명대,2021
7070,2,2.992296,3,0700713N713,울산농소,2021
7071,2,3.685612,3,0800N42,광주시가지,2021
7072,2,4.258013,3,0800Y20N025,동두천중앙로,2021
7073,2,3.569107,3,0800823,부천역,2021


In [20]:
df = df[df['RESEARCH_DATE'] >= 2019]
df.columns = ['buld_gbn','level_no_out','region_cd','region_nm','research_date','ret_on_invest']

In [21]:
df

,buld_gbn,level_no_out,region_cd,region_nm,research_date,ret_on_invest
609,2,3,1500N94,마산역버스터미널,2019,2.333390
610,2,3,1500N96,창원시청,2019,3.368050
611,2,3,1500N97,창원역,2019,0.862489
612,4,3,1111N103,명동,2019,8.092363
613,4,3,1111104,종로,2019,7.214293
...,...,...,...,...,...,...
7069,2,3,0300311,계명대,2021,7.404620
7070,2,3,0700713N713,울산농소,2021,6.567498
7071,2,3,0800N42,광주시가지,2021,9.298836
7072,2,3,0800Y20N025,동두천중앙로,2021,7.366762


In [22]:
df.to_csv('./invest_rate_year.csv', index=False, encoding='utf-8')

###### test

In [22]:
DF = pd.read_csv('./etc_income_ratio.csv')
DF

,buld_gbn,etc_income,level_no_out,region_cd,region_nm,research_date
0,2,0.000000,3,0200Y19N005,남항동,201901
1,2,0.112767,1,0300000,대구,201901
2,2,0.083760,1,0400000,인천,201901
3,2,0.000000,3,0500511,금남로/충장로,201901
4,2,0.000000,3,0500514,상무지구,201901
...,...,...,...,...,...,...
10597,3,0.000000,3,3100111,삼산동,202201
10598,3,0.000000,3,3100115,울산농소,202201
10599,3,0.000000,1,3600000,세종,202201
10600,3,0.000000,3,4100111,고양시청,202201


In [23]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10602 entries, 0 to 10601
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   buld_gbn       10602 non-null  int64  
 1   etc_income     10602 non-null  float64
 2   level_no_out   10602 non-null  int64  
 3   region_cd      10602 non-null  object 
 4   region_nm      10602 non-null  object 
 5   research_date  10602 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 497.1+ KB


### DB 넣기

In [25]:
# module
# import pysftp # 압축 풀기
import psycopg2 # db connect
import pandas as pd
import numpy  as np
import zipfile  # 압축 풀기 
import datetime # 시간 출력

In [27]:
# 데이터 프레임 관련
import numpy  as np
import pandas as pd

# API 호출 관련
import json
import requests

In [30]:
# DB Connect
conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)

In [33]:
# 데이터 프레임 생성
DF_final = pd.DataFrame()
table = 'm1.income_rate_year'

# 파라미터, url 설정
page = 1
perPage = 1000000
serviceKey = 'sJuqUegKT1Q0R4IRDqoJklMK8xDxru%2BrNu%2BK5JXRh1mbeH4tFTWwQYWVK3xdbbJCCe14u5bQyl43S28%2F5TtfrA%3D%3D'
url = f'https://api.odcloud.kr/api/CommercialRealEstateLeaseTrendSvc/v1/getIncomRateOfReturnYear?page={page}&perPage={perPage}&serviceKey={serviceKey}'

try:
    response = requests.get(url)
    if response.status_code == 200: 
        # JSON 데이터 가져오기
        json = response.json()

        # json to dataFrame
        df = pd.json_normalize(json)

        # list to dataFrame
        DF = pd.DataFrame(df['data'][0])
        DF_final = pd.concat([DF_final, DF])
    else:
        response.close()
except Exception as e:
    print(e)

DF_final = DF_final[DF_final['RESEARCH_DATE'] >= '2019']
DF_final.columns = ['buld_gbn','income_rate','level_no_out','region_cd','region_nm','research_date']

# values 설정
val = '(' + ('%s,' * len(DF_final.columns))[:-1] + ')'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)
    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# insert
for j in range(0, len(DF_final), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = DF_final[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(DF_final))

2022-08-16 14:47:03.149426  :  0
2230


In [ ]:
# execute
execute_mogrify(conn, DF_final, table_nm, val)